# 协程

In [1]:
from inspect import getgeneratorstate


def simple_coroutine():
    print('-> coroutine started')
    x = yield 
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x7fa3181ea430>

In [4]:
next(my_coro)

-> coroutine started


In [5]:
getgeneratorstate(my_coro)

'GEN_SUSPENDED'

In [6]:
my_coro.send(42)

-> coroutine received: 42


StopIteration: 

In [23]:
getgeneratorstate(my_coro)

'GEN_CLOSED'

In [24]:
my_coro = simple_coroutine()
my_coro.send(1729)

TypeError: can't send non-None value to a just-started generator

In [25]:
getgeneratorstate(my_coro)

'GEN_CREATED'

In [26]:
def simple_coro2(a):
    print('-> Started: a =', a)
    b = yield a
    print('-> Received: b =', b)
    c = yield a + b
    print('-> Received: c =', c)

In [27]:
my_coro2 = simple_coro2(14)

In [28]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [29]:
next(my_coro2)

-> Started: a = 14


14

In [30]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [31]:
my_coro2.send(28)

-> Received: b = 28


42

In [32]:
my_coro2.send(99)

-> Received: c = 99


StopIteration: 

In [33]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

##  使用协程计算移动平均值

In [34]:
def make_average():    # 使用函数闭包计算移动平均值，函数make_average是一个高阶函数，应为它是一个把函数作为结果返回的函数
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

In [35]:
avg = make_average()
avg(10)

10.0

In [36]:
avg(11)

10.5

In [37]:
avg(12)

11.0

In [38]:
avg.__closure__

(<cell at 0x7fa31810eca0: int object at 0x954e60>,
 <cell at 0x7fa31810edf0: int object at 0x955220>)

In [39]:
avg.__closure__[0].cell_contents

3

In [40]:
avg.__code__.co_freevars

('count', 'total')

In [41]:
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [42]:
coro_avg = averager()
getgeneratorstate(coro_avg)

'GEN_CREATED'

In [43]:
next(coro_avg)

In [44]:
coro_avg.send(10)

10.0

In [45]:
coro_avg.send(30)

20.0

In [46]:
coro_avg.send(5)

15.0

In [47]:
from functools import wraps


def coroutine(func):
    """装饰器：向前执行到第一个`yield`表达式处，预激`func`"""
    @wraps(func)
    def primer(*args, **kwargs):
        gen = func(*args, **kwargs)
        next(gen)
        return gen
    return primer

In [48]:
@coroutine
def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count

In [49]:
coro_avg = averager()

In [50]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [51]:
coro_avg.send(10)

10.0

In [52]:
coro_avg.send(30)

20.0

In [53]:
coro_avg.send(5)

15.0

In [54]:
coro_avg = averager()
coro_avg.send(40)

40.0

In [55]:
coro_avg.send(50)

45.0

In [56]:
coro_avg.send('spam')

TypeError: unsupported operand type(s) for +=: 'float' and 'str'

In [59]:
getgeneratorstate(coro_avg)    # 为处理的异常会导致协程终止

'GEN_CLOSED'

In [60]:
coro_avg.send(60)

StopIteration: 

In [14]:
class DemoException(Exception):
    """为这次演示定义的异常类型"""
    
    
def demo_exc_handling():
    print('-> coroutine started')
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [15]:
exc_coro = demo_exc_handling()

In [16]:
next(exc_coro)

-> coroutine started


In [17]:
exc_coro.send(11)

-> coroutine received: 11


In [18]:
exc_coro.send(22)

-> coroutine received: 22


In [19]:
exc_coro.close()    
# 致使生成器在暂停的yield表达式处抛出GeneratorExit异常，如果生成器没有处理这个异常或抛出StopIteration异常（通常指运行到结尾），
# 则调用方不会报错。如果收到GeneratorExit异常，生成器一定不能产出值，否则解释器会抛出RuntimeError异常。生成器抛出的其他异常会向上
# 冒泡，传给调用方。

In [20]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [21]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [22]:
exc_coro.send(11)

-> coroutine received: 11


In [23]:
exc_coro.throw(DemoException)
# 致使生成器在暂停的yield表达式处抛出指定的异常。如果生成器处理了抛出的异常，代码会向前执行到下一个yield表达式，而产出的值会成为调用
# generator.throw方法得到的返回值。如果生成器没有处理抛出的异常，异常会向上冒泡，传到调用方的上下文中。

*** DemoException handled. Continuing...


In [24]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [72]:
exc_coro = demo_exc_handling()
next(exc_coro)

-> coroutine started


In [73]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [74]:
exc_coro.send(11)

-> coroutine received: 11


In [75]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [77]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

In [78]:
class DemoException(Exception):
    """为这次演示定义的异常类型"""
    
    
def demo_finally():
    print('-> coroutine started')
    try:
        while True:
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...')
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [79]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')


def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [80]:
coro_avg = averager()
next(coro_avg)

In [81]:
coro_avg.send(10)

In [82]:
coro_avg.send(30)

In [83]:
coro_avg.send(6.5)

In [84]:
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [86]:
coro_avg = averager()
next(coro_avg)

In [87]:
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

In [88]:
try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value

In [89]:
result

Result(count=3, average=15.5)

## 使用yield from

In [90]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1, 3):
        yield i

In [91]:
list(gen())

['A', 'B', 1, 2]

In [92]:
def gen():
    yield from 'AB'
    yield from range(1, 3)

In [93]:
g = gen()
g

<generator object gen at 0x7fa309a0c900>

In [94]:
next(g)

'A'

In [95]:
next(g)

'B'

In [96]:
getgeneratorstate(g)

'GEN_SUSPENDED'

In [97]:
next(g)

1

In [98]:
next(g)

2

In [99]:
next(g)

StopIteration: 

In [100]:
list(gen())

['A', 'B', 1, 2]

In [101]:
def chain(*iterables):
    for it in iterables:
        yield from it

In [102]:
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

['A', 'B', 'C', 0, 1, 2]

### 使用yield from计算平均值并输出统计报告

In [103]:
from collections import namedtuple

Result = namedtuple('Result', 'count average')

data = {'girls;kg': [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
        'girls;m': [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
        'boys;kg': [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
        'boys;m': [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]}


def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print('{:2} {:5} averaging {:.2f}{}'.format(result.count, group, result.average, unit))


def averager():
    total = 0.0
    count = 0
    average = None
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


def grouper(results, key):
    while True:
        results[key] = yield from averager()
        
        
def main(data):
    results = {}
    for key, values in data.items():
        group = grouper(results, key)
        next(group)             
        # 也可以调用group.send(None)，如果调用方调用next(..)函数或.send(None)方法，都要转交职责，在子生成器上调用next(..)函数
        for value in values:    # 把各个value传入grouper
            group.send(value)
        group.send(None)        # 把None传入grouper
    # print(results)
    report(results)
    

main(data)

 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


### yield from的意义

#### 简化的伪代码，等效于委派生成器中的RESULT = yield from EXPR语句(这里针对的是最简单的情况：不支持.throw(...）和.close(...)方法，而且只处理StopIteration异常)

#### 伪代码，等效于委派生成器中的RESULT = yield from EXPR语句

In [10]:
from collections import namedtuple

Event = namedtuple('Event', 'time proc action')


def taxi_process(ident, trips, start_time=0):
    """每次改变状态时创建事件，把控制权让给仿真器"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')
    # 出租车进程结束

In [11]:
taxi = taxi_process(ident=13, trips=2, start_time=0)

In [12]:
next(taxi)

Event(time=0, proc=13, action='leave garage')

In [13]:
taxi.send(_.time + 7)

Event(time=7, proc=13, action='pick up passenger')

In [14]:
taxi.send(_.time + 23)

Event(time=30, proc=13, action='drop off passenger')

In [15]:
taxi.send(_.time + 5)

Event(time=35, proc=13, action='pick up passenger')

In [16]:
taxi.send(_.time + 48)

Event(time=83, proc=13, action='drop off passenger')

In [17]:
taxi.send(_.time + 1)

Event(time=84, proc=13, action='going home')

In [18]:
taxi.send(_.time + 10)

StopIteration: 

## 相当精彩的离散事件仿真(DES)，揭示如何使用协程在单个线程中管理并发活动

In [26]:
import random
import collections
import queue
import argparse

DEFAULT_NUMBER_OF_TAXIS = 3
DEFAULT_END_TIME = 180
SEARCH_DURATION = 5
TRIP_DURATION = 20
DEPARTURE_INTERVAL = 5

Event = namedtuple('Event', 'time proc action')


def taxi_process(ident, trips, start_time=0):
    """每次改变状态时创建事件，把控制权让给仿真器"""
    time = yield Event(start_time, ident, 'leave garage')
    for i in range(trips):
        time = yield Event(time, ident, 'pick up passenger')
        time = yield Event(time, ident, 'drop off passenger')
    yield Event(time, ident, 'going home')
    # 出租车进程结束


def compute_duration(previous_action):
    """Compute action duration using exponential distribution"""
    if previous_action in ['leave garage', 'drop off passenger']:
        # new state is prowling
        interval = SEARCH_DURATION
    elif previous_action == 'pick up passenger':
        # new state is trip
        interval = TRIP_DURATION
    elif previous_action == 'going home':
        interval = 1
    else:
        raise ValueError('Unknown previous_action: %s' % previous_action)
    return int(random.expovariate(1/interval)) + 1    
    
    
class Simulator:
    
    def __init__(self, procs_map):
        self.events = queue.PriorityQueue()    # 优先级队列是离散事件仿真系统（单线程中管理并发活动）的基础构件
        self.procs = dict(procs_map)
        
    def run(self, end_time):
        """排定并显示事件，直到时间结束"""
        # 排定各辆出租车的第一个时间
        for _, proc in sorted(self.procs.items()):
            first_event = next(proc)
            self.events.put(first_event)
        
        sim_time = 0
        while sim_time < end_time:
            if self.events.empty():
                print('*** end of events ***')
                break
                
            current_event = self.events.get()
            sim_time, proc_id, previous_action = current_event
            print('taxi:', proc_id, proc_id * '  ', current_event)
            active_proc = self.procs[proc_id]
            next_time = sim_time + compute_duration(previous_action)
            try:
                next_event = active_proc.send(next_time)
            except StopIteration:
                del self.procs[proc_id]
            else:
                self.events.put(next_event)
        else:
            msg = '*** end of simulation time: {} events pending ***'
            print(msg.format(self.events.qsize()))

In [27]:

def main(end_time=DEFAULT_END_TIME, num_taxis=DEFAULT_NUMBER_OF_TAXIS,
         seed=None):
    """Initialize random generator, build procs and run simulation"""
    if seed is not None:
        random.seed(seed)  # get reproducible results

    taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL)
             for i in range(num_taxis)}
    sim = Simulator(taxis)
    sim.run(end_time)

In [29]:
main(seed=3)

taxi: 0  Event(time=0, proc=0, action='leave garage')
taxi: 0  Event(time=2, proc=0, action='pick up passenger')
taxi: 1    Event(time=5, proc=1, action='leave garage')
taxi: 1    Event(time=8, proc=1, action='pick up passenger')
taxi: 2      Event(time=10, proc=2, action='leave garage')
taxi: 2      Event(time=15, proc=2, action='pick up passenger')
taxi: 2      Event(time=17, proc=2, action='drop off passenger')
taxi: 0  Event(time=18, proc=0, action='drop off passenger')
taxi: 2      Event(time=18, proc=2, action='pick up passenger')
taxi: 2      Event(time=25, proc=2, action='drop off passenger')
taxi: 1    Event(time=27, proc=1, action='drop off passenger')
taxi: 2      Event(time=27, proc=2, action='pick up passenger')
taxi: 0  Event(time=28, proc=0, action='pick up passenger')
taxi: 2      Event(time=40, proc=2, action='drop off passenger')
taxi: 2      Event(time=44, proc=2, action='pick up passenger')
taxi: 1    Event(time=55, proc=1, action='pick up passenger')
taxi: 1    Eve